# Importing Libraries

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,BatchNormalization,Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img,img_to_array

# Setting Directory

In [4]:
train_directory='./train'
val_directory='./valid'
test_directory='./test'

In [5]:
train_datagen=ImageDataGenerator(rescale=1/255)
val_datagen=ImageDataGenerator(rescale=1/255)
test_datagen=ImageDataGenerator(rescale=1/255)

In [6]:
train_generator=train_datagen.flow_from_directory(train_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

val_generator=val_datagen.flow_from_directory(val_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

test_gemerator=test_datagen.flow_from_directory(test_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse',batch_size=256)

Found 43622 images belonging to 300 classes.
Found 1500 images belonging to 300 classes.
Found 1500 images belonging to 300 classes.


# Classes of Birds 

In [7]:
train_generator.class_indices

{'AFRICAN CROWNED CRANE': 0,
 'AFRICAN FIREFINCH': 1,
 'ALBATROSS': 2,
 'ALEXANDRINE PARAKEET': 3,
 'AMERICAN AVOCET': 4,
 'AMERICAN BITTERN': 5,
 'AMERICAN COOT': 6,
 'AMERICAN GOLDFINCH': 7,
 'AMERICAN KESTREL': 8,
 'AMERICAN PIPIT': 9,
 'AMERICAN REDSTART': 10,
 'ANHINGA': 11,
 'ANNAS HUMMINGBIRD': 12,
 'ANTBIRD': 13,
 'ARARIPE MANAKIN': 14,
 'ASIAN CRESTED IBIS': 15,
 'BALD EAGLE': 16,
 'BALD IBIS': 17,
 'BALI STARLING': 18,
 'BALTIMORE ORIOLE': 19,
 'BANANAQUIT': 20,
 'BANDED BROADBILL': 21,
 'BANDED PITA': 22,
 'BAR-TAILED GODWIT': 23,
 'BARN OWL': 24,
 'BARN SWALLOW': 25,
 'BARRED PUFFBIRD': 26,
 'BAY-BREASTED WARBLER': 27,
 'BEARDED BARBET': 28,
 'BEARDED BELLBIRD': 29,
 'BEARDED REEDLING': 30,
 'BELTED KINGFISHER': 31,
 'BIRD OF PARADISE': 32,
 'BLACK & YELLOW bROADBILL': 33,
 'BLACK FRANCOLIN': 34,
 'BLACK SKIMMER': 35,
 'BLACK SWAN': 36,
 'BLACK TAIL CRAKE': 37,
 'BLACK THROATED BUSHTIT': 38,
 'BLACK THROATED WARBLER': 39,
 'BLACK VULTURE': 40,
 'BLACK-CAPPED CHICKADEE': 41,

# Transfer Learning Using RESNET_101

In [9]:
from tensorflow.keras.applications import ResNet101V2
convlayer=ResNet101V2(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in convlayer.layers:
    layer.trainable=False

171327488/171317808 [==============================] - 2s 0us/step


In [13]:
model=Sequential()
model.add(convlayer)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(300,activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101v2 (Functional)     (None, 7, 7, 2048)        42626560  
_________________________________________________________________
dropout_3 (Dropout)          (None, 7, 7, 2048)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 100352)            401408    
_________________________________________________________________
dense_3 (Dense)              (None, 2048)              205522944 
_________________________________________________________________
batch_normalization_4 (Batch (None, 2048)              8192      
_________________________________________________________________
activation_2 (Activation)    (None, 2048)             

In [14]:
opt=tf.keras.optimizers.Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)

In [15]:
history=model.fit(train_generator,validation_data=val_generator,
         epochs=5)

Epoch 1/5
  9/171 [>.............................] - ETA: 1:11:04 - loss: 5.8122 - accuracy: 0.0378

KeyboardInterrupt: 

### Training with lower learning rate for last iteration

In [ ]:
opt=tf.keras.optimizers.RMSprop(lr=0.0001)
model.compile(loss='sparse_categorical_crossentropy',metrics=['accuracy'],optimizer=opt)
history_1=model.fit(train_generator,validation_data=val_generator,
         epochs=1)

# Accuracy Plot

In [ ]:
plt.plot(history.history['accuracy'],c='red')
plt.plot(history.history['val_accuracy'],c='green')
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train','validation'],loc='lower right')

# Loss Plot

In [ ]:
plt.plot(history.history['loss'],c='red')
plt.plot(history.history['val_loss'],c='green')
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train','validation'],loc='upper right')

# Test Accuracy 

In [ ]:
model.evaluate(test_gemerator)

# Prediction

Function for predicting bird class

In [ ]:
dic=train_generator.class_indices
icd={k:v for v,k in dic.items()}
def output(location):
    img=load_img(location,target_size=(224,224,3))
    img=img_to_array(img)
    img=img/255
    img=np.expand_dims(img,[0])
    answer=model.predict_classes(img)
    probability=round(np.max(model.predict_proba(img)*100),2)
    #print ('Bird Is',icd[answer[0]], 'With probability',probability)
    print (probability, ' % chances are there that the Bird Is',icd[answer[0]])

### Belted Kingfisher

In [ ]:
img='../input/anurag-mishra/belted.jpg'
pic=load_img('../input/anurag-mishra/belted.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)

### ARARIPE MANAKIN 

In [ ]:
img='../input/fdjn-vfvjkfd-v/hg.jpg'
pic=load_img('../input/fdjn-vfvjkfd-v/hg.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)

### BANANAQUIT

In [ ]:
img='../input/fj-fdjvdk-vmdvdjl/bananan.jpg'
pic=load_img('../input/fj-fdjvdk-vmdvdjl/bananan.jpg',target_size=(224,224,3))
plt.imshow(pic)
output(img)